In [37]:
import re
from pathlib import Path
import pandas as pd

data_folder = Path("C:/Users/Jing Li/Desktop/cv2db/project/Inputs")
# data1_to_open = data_folder / "All Resumes_Angela Bowden.txt"
# data2_to_open = data_folder / "PEPFAR All Resumes_10.2+11.3.txt"
data3_to_open = data_folder / "PEPFAR All Resumes_10.2+11.3 (sparse).txt"


# with open(data1_to_open) as f: 
# 	data1 = f.read() #data1 is a string

# with open(data2_to_open) as f: 
# 	data2 = f.read()
    
with open(data3_to_open) as f: 
	data = f.read()

In [46]:
# Split: name and title

pattern_ppl = r"[^\n]+\nCareer Summary"
pattern_name = r"\s–\s"

ppls_noname = re.split(pattern_ppl, data)
ppls_noname = list(filter(None, ppls_noname))

name_title = re.findall(pattern_ppl, data)

name = [re.split(pattern_name, i)[0] for i in name_title]
title = [re.split(pattern_name, i)[1] for i in name_title]
title1 = [re.sub(r'\nCareer Summary', '', i) for i in title] #remove '\nCareer Summary' from title

21


In [158]:
# Split: career summary, ETC (Education, Training, and Certifications), Work Experience

#Career Summary
pattern_crr_smmry = r"\nEducation, Training, and Certifications"
crr_smmry = [re.split(pattern_crr_smmry, i)[0] for i in ppls_noname]
crr_smmry = [i.replace("\n", " ") for i in crr_smmry] #career summary
ppls1 = [re.split(pattern_crr_smmry, i)[1] for i in ppls_noname]

#ETC
pattern_ETC = r"\nWork Experience"
ETC = [re.split(pattern_ETC, i)[0] for i in ppls1] #Education, Training, and Certifications
ppls2 = [re.split(pattern_ETC, i)[1] for i in ppls1]

#Work Experience
pattern_Experience = r"(\n(Publications and Awards|Awards\/Honors|Publications|Research and Publications))"
ppls2_list = [re.split(pattern_Experience, i) for i in ppls2]
for i in ppls2_list:
    i.extend(['N/A','N/A','N/A'])
wrk_exp = [i[0] for i in ppls2_list] #Work Experience
pub = [i[3] for i in ppls2_list] #Hornors/Awards/Publications

#Extract Qualifications and Experience from Work Experience
QElength = [2,2,3,2,3, 3,1,2,3,3, 4,2,3,2,3, 2,3,2,3,3, 3]
pattern_QE = [r'Qualifications and Experience([^•]*•){' + str(i) + '}\n[•–·(Senior)(Country)]|[a-zA-Z ]{70}' for i in QElength]
wrk_exp = [re.findall]
(Qualifications and Experience)([^•]*•).+


# further seperate work experience
pattern_exp = r"\n[^\n]+\d{4}\s?[–-]"
wrk_exp_list = [re.split(pattern_exp, i) for i in wrk_exp]
wrk_exp_list1 = [re.findall(pattern_exp, i) for i in wrk_exp]
for i in range(len(wrk_exp_list)):
    wrk_exp_list[i].remove("")
    wrk_exp_list[i] = [wrk_exp_list1[i][j] + wrk_exp_list[i][j] for j in range(len(wrk_exp_list[i]))]


In [160]:
# Build a simple dataframe to store the chunked information

df1 = pd.DataFrame({"Name":name,
                   "Title": title1,
                   "Career Summary": crr_smmry,
                   "ETC": ETC,
                   "Work Experience": wrk_exp_list,
                   "Hornors/Awards/Publications": pub})

In [161]:
df1["Work Experience"][0]

['\nProgram Manager May 2019 – Present\nDSFederal, Inc., Rockville, MD\nProject: Food and Drug Administration / Integrated Services\n• responsibilities include managing training projects under FDA training integration blanket purchase agreement. Duties include Providing a technical expertise proposals, managing all aspects of training lifecycle development, knowledge management, and human capital development.',
 '\nSr. ISD and Training Manager October 2017 – May 2019\nHendall Inc., Rockville, MD Project: Centers for Medicaid and Medicare / Training Support\n• Work closely with SMEs and stakeholders to ensure the accuracy and quality of all course content.\n• Manage a dynamic training team that includes graphic designers, course developers, and technical writers to produce deliverables.\n• Employ the ADDIE Model and performance-based lean ISD processes to produce high-quality\nweb-based and streaming training solutions.\n• Oversee the progress of operations, including:\no Conducting nee